In [ ]:
import pandas as pd
from fyers_helper import prepare_data, load_stock_data
import datetime as dt

interval = "5"

nifty200_df = pd.read_csv('nifty50.csv')
tickers = [ f'NSE:{n}-EQ' for n in nifty200_df.Symbol.tolist()]

data_path = "../data5m"

end_date = dt.datetime.now()
start_date = dt.datetime(2015, 1, 1)

file_paths = prepare_data(tickers, interval, start_date=start_date, end_date=end_date, path=data_path, overwrite=False)
loaded_data_1D = load_stock_data(file_paths, data_path, interval)

df = pd.concat(loaded_data_1D, names=["Stock", "Date"]).reset_index()
df = df[(df.Date.dt.time >= dt.time(9,15)) & (df.Date.dt.time < dt.time(15,30))]

In [ ]:
from lib import Bank, PositionManager, generate_tearsheet
import talib as ta
from tqdm.notebook import tqdm


def backtest(_df, pm: PositionManager, params, show_pb=False):
    max_positions = params['max_positions']
    fast_ma = int(params['fast_ma'])
    slow_ma = int(params['slow_ma'])
    trend_filter_ma = params['trend_filter_ma']
    atr_period = params['atr_period']
    start_date = params['start_date']
    end_date = params['end_date']
    initial_capital = params['initial_capital']

    print(f"fast_ma - {fast_ma}, slow_ma - {slow_ma}")

    _df['FastMA'] = _df.groupby('Stock')['Close'].transform(lambda x: x.rolling(fast_ma).mean())
    _df['SlowMA'] = _df.groupby('Stock')['Close'].transform(lambda x: x.rolling(slow_ma).mean())
    _df['TrendFilterMA'] = _df.groupby('Stock')['Close'].transform(lambda x: x.rolling(trend_filter_ma).mean())

    def calc_atr(group):
        group = group.copy()
        group['ATR'] = ta.ATR(group['High'], group['Low'], group['Close'], atr_period)
        return group

    _df = _df.groupby('Stock', group_keys=False).apply(calc_atr)

    _df["NextTime"] = _df.groupby('Stock')['Date'].shift(-1)
    _df["NextOpen"] = _df.groupby('Stock')['Open'].shift(-1)
    _df["PrevFastMA"] = _df.groupby('Stock')['FastMA'].shift()
    _df["PrevSlowMA"] = _df.groupby('Stock')['SlowMA'].shift()

    _df = _df.dropna().reset_index(drop=True)
    _df = _df[(_df.Date >= start_date) & (_df.Date < end_date)].reset_index()

    _df['signal'] = 0
    _df.loc[(_df.Close > _df.TrendFilterMA) & (_df.FastMA > _df.SlowMA) & (_df.PrevFastMA < _df.PrevSlowMA), 'signal'] = 1
    _df.loc[(_df.Close < _df.TrendFilterMA) & (_df.FastMA < _df.SlowMA) & (_df.PrevFastMA > _df.PrevSlowMA), 'signal'] = -1

    signals = _df[_df.signal != 0]

    if show_pb:
        pb = tqdm(total=_df.Date.nunique(), desc="Backtesting...")

    for idx, group in _df.groupby('Date'):
        day_signals = signals[signals.Date == idx]
        for signal in day_signals.itertuples():
            if pm.get_position(signal.Stock) is None:
                capital = round(pm.bank.total_capital() / max_positions, 2)
                position = pm.new_position(signal.Stock, signal.NextTime, signal.NextOpen, capital)
                if position is not None:
                    position.tp = position.avg_entry_price + signal.ATR * 2
                    position.sl = position.avg_entry_price - signal.ATR * 2


        for position in pm.get_active_positions():
            stock = group[group.Stock == position.stock]
            if (stock.Close.iloc[0] >= position.tp) or (stock.Close.iloc[0] <= position.sl):
                pm.close_position(position.stock, idx, stock.Close.iloc[0])

        if show_pb:
            pb.update(1)

    if show_pb:
        pb.close()
    
    return generate_tearsheet(initial_capital, pm)


In [ ]:
from lib import optimize
from hyperopt import hp

CONSTANT_PARAMS = {
    'initial_capital': 100000,
    "max_positions": 10,
    "trend_filter_ma": 200,
    "atr_period": 40,
    'start_date': '2025-01-01',
    'end_date': '2025-05-01'
}


if __name__ == '__main__':
    
    space = {
        "fast_ma": hp.quniform('fast_ma', 10, 50, 5),
        "slow_ma": hp.quniform('slow_ma', 60, 180, 20),
    }

    best = optimize(backtest, df, space, CONSTANT_PARAMS)
    print(best)
    

In [ ]:
from lib import Bank, PositionManager, show_equity_curve

params = {
    'initial_capital': 100000,
    "max_positions": 10,
    "trend_filter_ma": 200,
    "atr_period": 40,
    'start_date': '2025-05-01',
    'end_date': '2025-06-01',
    'fast_ma': 40,
    'slow_ma': 140
}


bank = Bank(params['initial_capital'])
pm = PositionManager(bank)

tearsheet, trades = backtest(df.copy(), pm, params, show_pb=True)
print(pd.DataFrame({
    "Metrics": tearsheet.keys(),
    "Values": tearsheet.values(),
}))
show_equity_curve(trades)